## Day 16: Proboscidea Volcanium

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day16/puzzle.ipynb)

In [51]:
#!value --name sampleRaw
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II

### Parsing

In [ ]:
#r "nuget:Farkle, 6.3.2"
open Farkle
open Farkle.Builder
open Farkle.Builder.Regex

Installing Packages Farkle...............

In [ ]:
type Valve = { Id : string; FlowRate: int; LeadsTo: string list }

type Network = Map<string, Valve>

In [ ]:
#load "../common/common.fsx"

let private valve = 
    chars Letter 
    |> atLeast 1 
    |> terminal "Valve" (T(fun _ x -> x.ToString()))

let private flowRate = Terminals.int "FlowRate"

let private valveInfo = "Info" ||= [
    !& "Valve" .>>. valve 
        .>> "has flow rate=" .>>. flowRate 
        .>> "; tunnel leads to valve" .>>. valve 
            => fun valve rate valveTo 
                -> { Id = valve; FlowRate = rate; LeadsTo = [valveTo] }
    !& "Valve" .>>. valve 
        .>> "has flow rate=" .>>. flowRate 
        .>> "; tunnels lead to valves" .>>. (sepBy1 (literal ",") valve) 
            => fun valve rate valvesTo 
                -> { Id = valve; FlowRate = rate; LeadsTo = valvesTo }
]

let parser = RuntimeFarkle.build valveInfo
let parse = RuntimeFarkle.parseString parser >> Result.get


In [ ]:
#!share sampleRaw --from value


let sampleNetwork = 
    Pattern1.read parse sampleRaw
    |> Seq.map (fun x -> x.Id, x)
    |> Map.ofSeq


In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.Mermaid,1.0.0-beta.22405.1"

Installed Packages Microsoft.DotNet.Interactive.Mermaid, 1.0.0-beta.22405.1

In [ ]:
open Microsoft.DotNet.Interactive.Mermaid

let toGraph (network: Network) = 
    let normalized = 
        network
        |> Map.toSeq
        |> Seq.collect (fun (from,v) -> v.LeadsTo |> List.map(fun to' -> from, to'))
        |> Seq.distinctBy (fun (from, to') -> max from to', min from to')
        |> Seq.map (fun (from, to') -> network[from],network[to'])
    let mermaidLines = 
        normalized
        |> Seq.map (fun (x,y) -> $"    {x.Id}[{x.Id} {x.FlowRate}] --- {y.Id}[{y.Id} {y.FlowRate}]")
        |> Seq.append [|"graph LR"|]
    String.Join("\n", mermaidLines)
    |> MermaidMarkdown

In [ ]:
sampleNetwork |> toGraph

In [ ]:
#!value --name actualRaw --from-file ./data_actual.txt

In [ ]:
#!share actualRaw --from value
let actualNetwork = 
    Pattern1.read parse actualRaw
    |> Seq.map (fun x -> x.Id, x)
    |> Map.ofSeq


### Part 1

This notebook relies on a breadth-first search algorithm implementation from the `bfs.fsx` script file. If no target condition was specified, or a target was not found, the algorithm returns all the paths had been gone through until the traverse process was considered no longer possible.

In this problem we have a graph of something more than valves connected with tunnels. It is more like a graph of decicions taken during each minute. We are not interested in any particular final decision, but want to traverse all possible decision paths reachable in 30 minutes.

In [ ]:
type State = { Time: int; TimeLeft: int; Network: Network; Current: Valve; Open: list<string>; Flow: int; MaxSkippedRate: int } 

In `Flow` we store the flow we will accumulate after all 30 minutes pass. `MaxSkippedRate` is going to be handy for some optimization (1) in the adjacent states generation process. The idea is that when we decide *not* to open the current valve, there is no point in opening any valve which has *lower* flow rate later on. Some other optimizations that reduce the number of possible states will be:

2. It does not make sense to ramble around the cave when all the valves have been opened, even though we still have time left.
3. It does not make sense to continue the search process when there is no time left.
4. The number of states can be significantly reduced by skipping through zero-rate rooms.

In [ ]:
#load "../common/bfs.fsx"
open Bfs.Custom

let adj : Adjacency<State> =
    let rec adj' visited0 state =
        if (state.TimeLeft <= 0) // 3
        then []
        elif ((state.Open |> List.length) = (state.Network |> Map.count)) // 2
        then []
        else 
        [
            let closed = state.Current.FlowRate > 0 && (state.Open |> List.contains state.Current.Id |> not)
            if (closed && state.Current.FlowRate <= state.MaxSkippedRate) // 1
            then
                yield {state with Flow = state.Flow + state.Current.FlowRate * (state.TimeLeft - 1); Open = state.Current.Id::state.Open; TimeLeft = state.TimeLeft - 1; Time = 1}

            yield!
                state.Current.LeadsTo
                |> List.filter (fun leadsTo -> Set.contains leadsTo visited0 |> not)
                |> List.map (fun leadsTo -> Map.find leadsTo state.Network)
                |> List.collect(fun leadsTo ->
                            let leadsToState = {
                                state with MaxSkippedRate = max state.MaxSkippedRate leadsTo.FlowRate
                                           Current = leadsTo
                                           TimeLeft = state.TimeLeft - 1
                            }
                            if (leadsTo.FlowRate > 0)
                            then [leadsToState]
                            else // 4
                                adj' (Set.add state.Current.Id visited0) leadsToState
                                |> List.sortByDescending (fun x -> x.TimeLeft)
                                |> List.distinctBy (fun x -> x.Current.Id)
                ) |> List.map (fun leadsTo -> {leadsTo with Time = state.TimeLeft - leadsTo.TimeLeft})
        ]
    adj' Set.empty 

In [ ]:

let target : Bfs.Custom.Target<State> =
    fun state -> false
    
let settings =
    {VisitedKey = fun state -> state.Current.Id, state.Flow, state.Open}

let traverse (network:Network) (start:string) (time:int) = 
    let startValve = network[start]
    let state = { MaxSkippedRate = 0; Time = 0; TimeLeft = time; Network = network; Current = startValve; Open = []; Flow = 0 } 
    Bfs.Custom.findPath settings { Adjacency = adj } state target 


In [ ]:
#!time

let sampleMax1 = 
    let (NotFound(states)) = traverse sampleNetwork "AA" 30
    states 
    |> Seq.map List.head
    |> Seq.map (fun s -> s.Flow)
    |> Seq.max
    
sampleMax1

1651

Wall time: 59.4051ms

In [ ]:
#!time

let actualMax1 = 
    let (NotFound(states)) = traverse actualNetwork "AA" 30
    states 
    |> Seq.map List.head
    |> Seq.map (fun s -> s.Flow)
    |> Seq.max

actualMax1

1828

Wall time: 9203.4414ms

### Part 2

For part two we will use the fact then the BFS implementation we rely on returns not only full paths to final states, but also incomplete paths prior to those. We can run the algorithm for one valve-opener once and then scan the cartesian square for not-intersecting open valve sets, as it would not make sense to open a valve twice.

Before that, however, it can be noticed that there will be many paths to the same open valve sets. We can take only one of them, with the best flow.

In [ ]:
let part2 network = 
    let (NotFound(states)) = traverse network "AA" 26
    let best =
        states
        |> Seq.map List.head
        |> Seq.sortByDescending (fun x-> x.Flow)
        |> Seq.map (fun x -> x, x.Open |> Set.ofList)
        |> Seq.distinctBy snd
    let pairs = Seq.allPairs best best
    pairs
    |> Seq.filter(fun ((s1,open1), (s2, open2))-> Set.intersect open1 open2 |> Set.isEmpty)
    |> Seq.map(fun ((s1,open1), (s2, open2)) -> s1.Flow + s2.Flow)
    |> Seq.max

In [ ]:
#!time 

part2 sampleNetwork

1707

Wall time: 457.7294ms

In [ ]:
#!time 

part2 actualNetwork

2292

Wall time: 22380.2628ms